In [1]:
!pip install requests beautifulsoup4 pandas numpy matplotlib


Importando bibliotecas

In [2]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import numpy as np


Analise para extração de dados 

Extração de dados de múltiplas páginas 

In [8]:
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import numpy as np

# Ignora o aviso de InsecureRequestWarning
warnings.simplefilter('ignore', InsecureRequestWarning)

# Função para analisar uma página e extrair dados
def analisa_pagina(url):
    # Faz a requisição da página
    pagina = rq.get(url, verify=False)
    # Cria o analisador de HTML
    analisador_pagina = bs(pagina.content, 'html.parser')
    
    # Lista para armazenar informações de celulares
    celulares = []
    
    # Encontra todos os elementos de produtos
    for x in analisador_pagina.find_all(class_='ProductCard_ProductCard_Body__bnVUn'):
        imagemCelular = x.find(class_="ProductCard_ProductCard_Image__4v1sa").find('img')['src']
        modeloCelular = x.find(class_="ProductCard_ProductCard_NameWrapper__45Z01").text
        precoCelular = x.find(class_="Text_Text__ARJdp Text_MobileHeadingS__HEz7L").text
        
        # Adiciona as informações extraídas à lista de celulares
        celulares.append({
            'imagem': imagemCelular,
            'modelo': modeloCelular,
            'preço': precoCelular
        })
    
    return celulares

# URL base para busca de dados
url_base = "https://www.bondfaro.com.br/celular?page="

# Extração de dados de múltiplas páginas
celulares_total = []

# Loop para percorrer as páginas (ajuste o número conforme necessário)
for x in range(1, 8):
    url_pagina = url_base + str(x)
    celulares_total.extend(analisa_pagina(url_pagina))

# Criação de DataFrame
df = pd.DataFrame(celulares_total)

# Limpeza e conversão da coluna de preços
df['preço'] = (
    df['preço']
    .replace({'R\$': '', '\.': '', ',': '.'}, regex=True)  # Remove 'R$', pontos e troca vírgula por ponto
    .replace('', np.nan)  # Substitui strings vazias por NaN
    .astype(float)  # Converte para float
)

# Adiciona uma coluna de data e hora
df['Data_Hora_Local'] = pd.to_datetime(datetime.now())

# Exibe o DataFrame para visualização
print(df)


                                                imagem  \
0    https://i.zst.com.br/thumbs/45/38/33/-12396073...   
1    https://i.zst.com.br/thumbs/45/a/30/-949891967...   
2    https://i.zst.com.br/thumbs/45/38/14/-13013616...   
3    https://i.zst.com.br/thumbs/45/19/31/-11139722...   
4    https://i.zst.com.br/thumbs/45/a/30/-949891967...   
..                                                 ...   
166  https://i.zst.com.br/thumbs/45/21/1/-105908513...   
167  https://i.zst.com.br/thumbs/45/10/30/-98922075...   
168  https://i.zst.com.br/thumbs/45/20/f/-140285897...   
169  https://i.zst.com.br/thumbs/45/36/38/-86644861...   
170  https://i.zst.com.br/thumbs/45/3d/32/-84527728...   

                                                modelo    preço  \
0             Celular Samsung Galaxy A55 5G 256GB 8 GB  2200.00   
1             Celular Samsung Galaxy S23 5G 256GB 8 GB  3649.99   
2       Celular Xiaomi Redmi Note 13 Pro 4G 256GB 8 GB  1403.00   
3                   Celular Apple i

In [9]:
df.to_csv('celulares_data.csv', index=False)
